In [1]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [2]:
import os
import numpy as np

from lib.cfg import Cfg
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

In [3]:
##  configuration options
el_charge = 1.6e-19
rho_beam_scale = 1
config_file = '../PicoPIC.json'

cfg = Cfg(config_file)

# update shape to get custom shaped images
# useful to get just part of frame
# or get frames, which has smaller shape than full frame
size=[0, 0, cfg.geometry_grid[0]-2, cfg.geometry_grid[1]-2]

timestamp=3.51e-10
show_grid=False
use_cache=False
cmap='terrain'

cmp = 'charge' # component

# color limits (WARNING: clim_estimation may works incorrectly)
clim_estimation = 1 # cfg.get_clim_estimation()
clim = [-clim_estimation, clim_estimation]
image_interpolation = 'nearest'

autoselect = True

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$\frac{V}{m}$'
cbar_bunch_density_axis_label = r'$m^{-3}$'

plot_name = r'$\mathbf{Something\enspace to\enspace Debug}$'

In [4]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(path = cfg.data_path,
                         fullframe_size=cfg.geometry_grid,
                         fpds=1, 
                         use_cache=use_cache,
                         verbose=True)
else:
    reader = H5Reader(str(os.path.join(cfg.data_path, 'data.h5')), use_cache=use_cache)
    reader.verbose = True

r_scale = (size[2] - size[0]) / cfg.geometry_grid[0]
z_scale = (size[3] - size[1]) / cfg.geometry_grid[1]

# define plot builder
plot = PlotBuilder(size[3] - size[1], size[2] - size[0],
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   
                   x_ticklabel_end=cfg.geometry_size[1] * z_scale, y_ticklabel_end=cfg.geometry_size[0] * r_scale,
                   tickbox=True, grid=show_grid, is_invert_y_axe=False,
                   aspect='equal', image_interpolation=image_interpolation, guess_number_ticks=20)

In [5]:
# get data
data = []

for probe in cfg.probes:
    frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
    if (probe.shape == 'rec') and (probe.size[0] == size[0]) and (probe.size[1] == size[1]) and(probe.size[2] == size[2]) and(probe.size[3] == size[3]):
        if probe.component == cmp: data = reader.get_frame(cmp, size, frame)

# try bigger frames, if autoselect enabled
if len(data) == 0 and autoselect:
    for probe in cfg.probes:
        frame = cfg.get_frame_number_by_timestamp(timestamp, probe.schedule)
        if (probe.shape == 'rec') and (probe.size[0] <= size[0]) and (probe.size[1] <= size[1]) and(probe.size[2] >= size[2]) and(probe.size[3] >= size[3]):
            if probe.component == cmp and len(data) == 0: data = reader.get_frame(cmp, probe.size, frame)[size[0]:size[2], size[1]:size[3]]

../picopic_result/charge/rec/0-127_0-1023/7.dat
Loading data set charge/rec:0-127_0-1023/7...done


In [6]:
# add subplots
plot.add_subplot_cartesian_2d(plot_name, 111, x_axe_label=x_axis_label, y_axe_label=y_axis_label)

# add timestamp to plot
plot.get_figure().suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.95)

# add images
plot.add_image(plot_name, data, cmap=cmap, clim=clim)

# add colorbars
plot.add_colorbar(plot_name, ticks=clim, title=cbar_axis_label)

plot.show()